# Data Privacy Concerns

## 1) Data Operations

This notebook contains the code for my fist medium article "NAME HERE" "LINK HERE"

As I explained in the article, my main motivation behind this research is to help understanding the factors related to data privacy concerns.

I first start with loading necessary libraries declaring required magic commands for this analysis.

In [ ]:
import pandas as pd
from plotnine import *
import matplotlib.pyplot as plt
import semopy as sm

#inline plotting for matplotlib in jupyter
%matplotlib inline

#to reload easily
%reload_ext autoreload
%autoreload 2

# I like ggplot style
plt.style.use('ggplot')

What I am doing below is defining the theoretical scope of this analysis. The "column_names" list is containing the names of the variables that I am going to use in this analysis. In the next chunk of code, I will be inserting this column into stata reader to make it return only these variables.

This is not an obligation. Especially for doing purely exploratory work, keeping the whole data might be beneficial. However, as you might realize by now, the designers of the survey made really terrible choice of variable naming. These are just some random numbers and letters. One can easily make mistakes working with them.

In [ ]:
# Column names that I want to keep
column_names = (["Q999","Q91_w7","Q92_w4","Q94_w4",
                 "Q93_w4","Q97_w4", "Q201A_w7","Q201B_w7",
                 "Q201C_w7","Q201D_w7","Q201E_w7","Q60A_w4","Q60B_w4",
                 "Q60C_w4","Q60D_w4","Q60E_w4","Q60F_w4","Q60G_w4",
                 "Q71A_w4","Q71B_w4","Q71C_w4","Q71D_w4","Q72A_w4",
                 "Q72B_w4","Q72C_w4","Q72D_w4","Q73A_w4","Q73B_w4",
                 "Q73C_w4","Q73D_w4","Q74A_w4","Q74B_w4","Q74C_w4",
                 "Q74D_w4","Q75A_w4","Q75B_w4","Q75C_w4","Q75D_w4"])


I am using pandas' stata reader. The important point when working with .dta files is you have both numeric versions and labelled versions of the data. In the labelled version one can see questions that are asked in the survey as well as the answers in human language. Whereas the numeric version only contains the numerical representation survey. So in order to understand what we are working with, we have to read both versions of the data. The iterator argument in the reader function allows us to do so.

Also, as I mentioned, I passed the column_names here into the function to make stata reader only read the columns in the file that are matching the list.

In [ ]:
# Calling the stata reader object
reader = (pd.read_stata(
    'df2.dta',
    columns=column_names,
    iterator=True)
    )

In [ ]:
# Reading the labels of the rows
value_labels = reader.value_labels()

In [ ]:
# Reading the column labels.
variable_labels = reader.variable_labels()

Below I am calling stata reader's read method with convert categoricals argument as False. This makes stata reader skip the labels and only return the numeric version of the data.

In [ ]:
df = reader.read(convert_categoricals=False)

So, below is a random sample of 5 rows from our dataset. As it stands, it's impossible to make sense of it. But with the help of value labels and variable labels we can make sense of it without referring to the questionnaire .pdf that added to this repo.

In [ ]:
df.sample(5)

In [ ]:
# showing the definitions of first three questions
list(variable_labels.items())[:3]

In [ ]:
# showing the value labels of first 3 questions
list(value_labels.items())[:3]

These two will aid us understanding what we are working with.
The first challenge arises here. For some reason variable_labels and value_labels are returning the whole dataset rather than the list of columns that I declared. I want the variable labels of the columns that I declared, not the whole variables. So in order to achieve that, I will use lambda function that go through variable labels dictionary and preserve elements only when the key is matching my column_names list.

In [ ]:
# Function to preserve the elements that are matching
dict_filter = lambda var, names: dict([ (element,var[element]) for element in var if element in names ])

labels = dict_filter(variable_labels, column_names)

Above code achieved what I aimed for. As I feel more natural with dataframes rather than dictionaries, I would like to make a neat data frame of variable names and questions that they are referring to. For that one, I loop over the dictionary items and construct a dataframe.

In [ ]:

variable = []
question = []
for key, value in labels.items():
    variable.append(key)
    question.append(value)

# Convert the dictionary to pandas dataframe
map_df = pd.DataFrame({"variable":variable, "question":question})

In [ ]:
map_df

After creating this reference data frame, now I would like to get rid of uninformative variable names by replacing them. So I constructed the dictionary below for this purpose.
The variable naming that I chose is actually rooted in the theory behind the survey design. If you check the questions above, you might realize that some set of questions are actually really close to each other in terms of what they are intended to measure. This is a latent variable construct. For instance, the questions that I renamed with open_ prefix are specially designed to measure the opennes of respondents personality. Similarly agrb_ prefix refers to set of questions intended to measure the respondents agreeableness. Those latent constructs together form the Big-Five personality traits taxonomy. Similar latent construct story holds true for pi_ prefix. With that set of questions, the concerns about personal information are measured.

In [ ]:
column_map = ({"Q999": "country", "Q91_w7": "gender",
               "Q92_w4": "age", "Q94_w4":"urbanization",  "Q93_w4": "region",
               "Q97_w4": "income", "Q201A_w7": "pi_misuse",
               "Q201B_w7": "pi_track", "Q201C_w7": "pi_safety", "Q201D_w7": "pi_share",
               "Q201E_w7": "pi_stranger", "Q60A_w4": "trst_prl", "Q60B_w4": "trst_pol",
               "Q60C_w4": "trst_prt", "Q60D_w4": "trst_leg", "Q60E_w4": "trst_plc",
               "Q60F_w4": "trst_gov", "Q60G_w4": "trst_eu", "Q71A_w4": "open_a",
               "Q71B_w4": "open_b", "Q71C_w4": "open_c", "Q71D_w4": "open_d",
               "Q72A_w4": "cons_a", "Q72B_w4": "cons_b", "Q72C_w4": "cons_c",
               "Q72D_w4": "cons_d", "Q73A_w4": "extr_a", "Q73B_w4": "extr_b",
               "Q73C_w4": "extr_c", "Q73D_w4": "extr_d", "Q74A_w4": "agrb_a",
               "Q74B_w4": "agrb_b", "Q74C_w4": "agrb_c", "Q74D_w4": "agrb_d",
               "Q75A_w4": "neur_a", "Q75B_w4": "neur_b", "Q75C_w4": "neur_c",
               "Q75D_w4": "neur_d"})

In [ ]:
df.rename(columns=column_map, inplace=True)

Great, now everything suits my liking.  Now we can explore what stories we have within.

In [ ]:
df.sample(5)

The first thing I would like to check is the shape of the data frame. We are talking about 11k observations with 40 variables. But I can see lots of NaNs in the dataframe. In the following lines I would like to make a report of missing values in order to decide my strategy to tackle it.

In [ ]:
df.shape

Below is the set of variables that are containing missing values. Remaining 31 variables are not containing any missing values. But the vital question is what is the amount of missing values? Just to check that I will first acquire percentage of valid values in all columns. The count() method counts the valid observations in the dataframe and len() returns the number of total observations regardless of their status. By dividing valid ones to the total length, I acquire the percentage of valid observations.

In [ ]:
list(df.columns[df.isnull().mean() != 0])

In [ ]:
# Percentage of valid observations for all variables
report_observations = pd.DataFrame(df.count() / len(df),
                                   columns=["percentage"]).\
    reset_index().\
    rename(columns={"index":"variable"})

In [ ]:
# Percentage of valid observations within the subset of columns that contain missing values
report_missing = report_observations[report_observations.percentage < 1]

In [ ]:
report_missing

Well, as concerns about personal information is my latent target variable in this analysis, it saddens me to see huge chunk of missing values are actually on those items that are measuring this latent variable. Let's make a sad visualisation of structure of missing values in this dataset.

In [ ]:
fig_1 = plt.figure(figsize=(9,5))
axes_1 = fig_1.add_axes([0,0,1, 1])
axes_1.set_title("Summary of Valid Observations")
axes_1.set_xlabel("Variable Names")
axes_1.set_ylabel("Percentage of Valid Observations")
for i in range(2,7):
    axes_1.annotate("Target Variable Items", xy=(i,0.2), xytext=(3.4,0.6,),
                    arrowprops =dict( shrink=0.005))
axes_1.bar(report_missing["variable"], report_missing["percentage"])

Since the items measuring concerns about personal information has the most missing values, that makes not much sense to consider other missing data strategies than dropping the all cases. The reason is, let's say, even if I use missing data imputation methods on other variables, the target variable would be still containing missing values.

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df["openness"] = (df["open_a"] + df["open_b"] + df["open_c"]+ df["open_d"]) / 4
df["conscientiousness"] = (df["cons_a"] + df["cons_b"] + df["cons_c"]+ df["cons_d"]) / 4
df["agreeableness"] = (df["agrb_a"] + df["agrb_b"] + df["agrb_c"]+ df["agrb_d"]) / 4
df["extraversion"] = (df["extr_a"] + df["extr_b"] + df["extr_c"]+ df["extr_d"]) / 4
df["neuroticism"] = (df["neur_a"] + df["neur_b"] + df["neur_c"]+ df["neur_d"]) / 4
df["pi_concerns"] = (df["pi_misuse"] + df["pi_track"] + df["pi_share"]+ df["pi_safety"] + df["pi_stranger"])  / 5
df["trust_in_inst"] = (df["trst_leg"] + df["trst_eu"] + df["trst_gov"]+ df["trst_plc"] + df["trst_pol"]+ df["trst_prl"] + df["trst_prl"]) / 7

In [ ]:
df.head()

In [ ]:
df.country.replace({1: "Denmark", 2: "Germany", 3:"Hungary", 4:"Spain"}, inplace=True)

In [ ]:
df.urbanization.replace({1: 'Big City',
                         2: 'Outer Big City',
                         3: 'Town/Small City',
                         4: 'Village',
                         5: 'Farm'}, inplace=True)

## 2) Exploration
In this section I would like to explore the relationships between certain variables. I personally don't find it useful to make bunch heatmap of correlations. To me exploration means more than that. At each level certain care must be given.

First thing that we have to keep in mind is that the most of the variables are discrete variables. This makes plotting and exploration phase little bit tricky. If you make a plot of discrete variable against another discrete variable, then what you'll end up with will not be informative plot. Let me quickly demonstrate the problem


In [ ]:
fig = plt.figure(figsize=(9,5))
axes = fig.add_axes([0,0,1, 1])
axes.set_title("A Very Very Bad Plot")
axes.set_ylabel("PI_Misuse (Discerete Variable)")
axes.set_xlabel("Trust in Legal System (Discerete Variable)")
plt.scatter(df["trst_leg"], df["pi_misuse"])

For this reason, when working with discrete variables, directly plotting variables against each other is not a smart idea. We need some transformations. What I am going to do during the exploration phase is, I just simply keep my X variable as is, while replacing Y with mean of the Y for each value in X. Following the example above, for each level in "trst_leg", I will calculate the mean of "pi_misuse" and plot them. Below plot is how it looks in action

In [ ]:
#Grouping by trst leg values and replacing pi_misuse with the mean of it
mask = df.groupby("trst_leg")["pi_misuse"].mean().reset_index().copy()

(
        ggplot(mask,
               aes(x="trst_leg", y="pi_misuse")) +
        geom_smooth(method="loess", se=True, color="darkred") +
        geom_point(color= "darkred") +
        xlab("Trust in Legal System") +
        ylab("Concern: Personal Information Misuse") +
        ggtitle("Relationship between Trust in Legal System \n and Personal Information Misuse")

)

I use plotnine library as it allows us to use almost all capabilities of R's legendary Ggplot package. The most beneficial part of this library is that it has super easy curve fitting function. As far as I am aware, seaborn also has some curve fitting functionality however plotnine's layered grammar of graphics is much easy to customise for me.

### 2.1) Demographic Variables on Personal Information Concerns

I will plot the demographics here

#### 2.1.A) Age on PI

The first demographic variable is Age. Seems like age is "associated" with increase in concerns about personal information up to around 70 years. Then the slope turns negative. However, it's important that we shouldn't make any causal conclusions here. It might be cohort or generation effects.

In [ ]:
mask = df.groupby("age")["pi_concerns"].mean().reset_index().copy()
mask.dropna(inplace=True)

(
        ggplot(mask,
               aes(x="age", y="pi_concerns")) +
        geom_smooth(method="loess",
                    se=True,
                    color="darkred") +
        geom_point(color="darkred") +
        xlab("Age") +
        ylab("Concerns About Personal Information ") +
        ggtitle("Relationship between Age \n and Concerns About Personal Information")

)

#### 2.1.B) Country on PI


In [ ]:
mask = df.groupby("country")["pi_concerns"].mean().reset_index().copy()
mask.dropna(inplace=True)

(
        ggplot(mask,
               aes(x="country", y="pi_concerns", fill="country")) +
        geom_col(stat="identity") +
        xlab("Trust in Legal System") +
        ylab("Concern: Personal Information Misuse") +
        ggtitle("Average Concerns About Personal Information Misuse Across Countries")

)

#### 2.1.C) Income on PI


In [ ]:
mask = df.groupby("income")["pi_concerns"].mean().reset_index().copy()
mask.dropna(inplace=True)
mask.income = mask.income.astype("int")

(
        ggplot(mask,
               aes(x="income", y="pi_concerns")) +
        geom_smooth(method="lm", se=True, color="darkred") +
        geom_point(color="darkred") +
        xlab("Personal Income") +
        ylab("Concern: Personal Information Misuse") +
        ggtitle("Relationship Between Personal Income \n and Concerns over Misuse of Personal Information")
)



In [ ]:
mask = df.groupby(["income", "gender"])["pi_concerns"].mean().reset_index().copy()
mask.dropna(inplace=True)
mask.income = mask.income.astype("int")

(
    ggplot(mask,
           aes(x="income", y="pi_concerns", color="factor(gender)")) +
    geom_smooth(method="lm", se=True) +
    geom_point() +
    xlab("Personal Income") +
    ylab("Concern: Personal Information Misuse") +
    scale_color_manual(name = "Gender",
                       values=["darkgreen", "darkred"],
                       labels=["Male", "Female"]) +
    ggtitle("Relationship Between Personal Income \n and Concerns over Misuse of Personal Information \n per Gender")
)




#### 2.1.D) Urbanization on PI


In [ ]:
mask = df.groupby(["urbanization"])["pi_concerns"].mean().reset_index().copy()
mask.dropna(inplace=True)

(
        ggplot(mask,
               aes(x="urbanization", y="pi_concerns", fill="urbanization")) +
        geom_col(stat="identity") +
        xlab("Urbanization") +
        ylab("Concerns About Misuse of Personal Information") +
        ggtitle("Relationship Between Urbanization \n and Concerns About Misuse of Personal Information")

)

### 2.2) Trust In Institutions on Concerns About Personal Information

In [ ]:
mask = df.groupby(["trust_in_inst"])["pi_concerns"].mean().reset_index().copy()
mask.dropna(inplace=True)

(
        ggplot(mask,
               aes(x="trust_in_inst", y="pi_concerns")) +
        geom_smooth(method="lm", se=True, color="darkred") +
        geom_point(color="darkred") +
        xlab("Trust In Institutions") +
        ylab("Concern: Personal Information Misuse") +
        ggtitle("Relationship Between Trust In Institutions \n and Concerns over Misuse of Personal Information")
)


### 2.3) Personality Traits on PI
#### 2.3.A) Openness on PI

In [ ]:
mask = df.groupby(["openness"])["pi_concerns"].mean().reset_index().copy()
mask.dropna(inplace=True)

(
        ggplot(mask,
               aes(x="openness", y="pi_concerns")) +
        geom_smooth(method="lm", se=True, color="darkred") +
        geom_point(color="darkred") +
        xlab("Openness") +
        ylab("Concern: Personal Information Misuse") +
        ggtitle("Relationship Between Openness \n and Concerns over Misuse of Personal Information")
)


#### 2.3.B) Extraversion on PI

In [ ]:
mask = df.groupby(["extraversion"])["pi_concerns"].mean().reset_index().copy()
mask.dropna(inplace=True)

(
        ggplot(mask,
               aes(x="extraversion", y="pi_concerns")) +
        geom_smooth(method="lm", se=True, color="darkred") +
        geom_point(color="darkred") +
        xlab("Extraversion") +
        ylab("Concern: Personal Information Misuse") +
        ggtitle("Relationship Between Extraversion \n and Concerns over Misuse of Personal Information")
)


#### 2.3.C) Neuroticism on PI


In [ ]:
mask = df.groupby(["neuroticism"])["pi_concerns"].mean().reset_index().copy()
mask.dropna(inplace=True)

(
        ggplot(mask,
               aes(x="neuroticism", y="pi_concerns")) +
        geom_smooth(method="loess", se=True, color="darkred") +
        geom_point(color="darkred") +
        xlab("Neuroticism") +
        ylab("Concern: Personal Information Misuse") +
        ggtitle("Relationship Between Neuroticism \n and Concerns over Misuse of Personal Information")
)


#### 2.3.D) Aggreableness on PI

In [ ]:
mask = df.groupby(["agreeableness"])["pi_concerns"].mean().reset_index().copy()
mask.dropna(inplace=True)

(
        ggplot(mask,
               aes(x="agreeableness", y="pi_concerns")) +
        geom_smooth(method="loess", se=True, color="darkred") +
        geom_point(color="darkred") +
        xlab("Agreeableness") +
        ylab("Concern: Personal Information Misuse") +
        ggtitle("Relationship Between Agreeableness \n and Concerns About Personal Information")
)

#### 2.3.E) Conscientiousness on PI

In [ ]:
mask = df.groupby(["conscientiousness"])["pi_concerns"].mean().reset_index().copy()
mask.dropna(inplace=True)

(
        ggplot(mask,
               aes(x="conscientiousness", y="pi_concerns")) +
        geom_smooth(method="loess", se=True, color="darkred") +
        geom_point(color="darkred") +
        xlab("Conscientiousness") +
        ylab("Concern: Personal Information Misuse") +
        ggtitle("Relationship Between Conscientiousness \n and Concerns About Personal Information")
)

Plotting dimensions can be increased. We can keep the X and Y axis and add further categories like below. But there are two problems associated with this exploration method. First, due to extreme amount of combination of the variables that can be used for plotting, going through all the combinations is neither viable nor cognitively helpful. Second, even if we can automatise the process of plotting, after certain number of plots it becomes really hard to build a story out of those plots.

In [ ]:
mask = df.groupby(["conscientiousness", "gender", "country"])["pi_concerns"].mean().reset_index().copy()
mask.dropna(inplace=True)

(
        ggplot(mask,
               aes(x="conscientiousness", y="pi_concerns", color="factor(gender)")) +
        geom_smooth(method="loess", se=True) +
        geom_point() +
        facet_grid("~country") +
        xlab("Conscientiousness") +
        ylab("Concern: Personal Information Misuse") +
        ggtitle("Relationship Between Conscientiousness \n and Concerns About Personal Information")
)

## 3) Regression Model of Concerns About Personal Information

To be able to work easily with higher dimensions we can statistical modelling. They are designed for this task. They can tell us about really complex relationships that we can't deduct only using our eyeballs. So, in the next section I will build a quick and dirty structural equation model that can sum up the whole story.

One remark that I have to make before advancing is, *I do not make any causal claims* using the structural equation model down below. Causality requires causal model of the subject matter and statistical model alone is not enough to talk about casual relationships. Throughout this notebook I used my sociological knowledge to guide the analysis, however, it is still far from being a sound argument for causality.

Second point that I want to draw attention is structural equation models are not the only way to attack the problem. It depends on what you wish to achieve. If your task is to explain the relationships, then parametric models can be really helpful in quantifying the relationships. They are white-box models and with certain tweaks they can fit data quite well to produce a robust story. If the target is to make predictions without caring too much about the size and direction of the relationships between the exposure and the target, then predictive inference engines like decision trees, random forest, gradient boosted trees etc are really helpful. As I am interested in the relationships rather than making predictions, I want to continue with structural equation model.

This study can be conducted using multivariate regression modelling as well. The additive indeces that I created to make plots can be used as predictor variables as well. This way the problem of multicollinearity wouldn't be a problem too.

In [ ]:
df.columns

In [ ]:
model_definition = """
    # MEASUREMENT MODEL

    ## Personality Taxonomy
    openness_l =~ open_a + open_b + open_c + open_d
    extraversion_l =~ extr_a + extr_b + extr_c + extr_d
    conscientiousness_l =~ cons_a + cons_b + cons_c + cons_d
    neuroticism_l =~ neur_a + neur_b + neur_c + neur_d
    agreeableness_l =~ agrb_a + agrb_b + agrb_c + agrb_d

    ## Political Trust
    trust_in_inst_l =~ trst_prl + trst_pol + trst_prt + trst_leg + trst_plc + trst_gov

    ## PI concerns
    pi_concerns_l =~ pi_misuse + pi_track + pi_safety + pi_share + pi_stranger

    # REGRESSION MODEL
    trust_in_inst_l ~ openness_l + extraversion_l + conscientiousness_l + neuroticism_l + agreeableness_l + gender + age + income
    pi_concerns_l ~ openness_l + extraversion_l + conscientiousness_l + neuroticism_l + agreeableness_l + trust_in_inst_l + gender + age + income
"""


In [ ]:
# Instantiate the model
model = sm.Model(model_definition)

# Fit the model using the data
model.fit(df)

# Show the results using the inspect method
model.inspect()

If below plot raises no graphviz module error, then you have to install python-graphviz from conda and restart the kernel

In [ ]:
from IPython.display import display
plot_results = sm.semplot(model, "plots/sem_results.png", )
display(plot_results)

In [ ]:
sm.report(model, "sem_report")